## Importing the required libraries

In [2]:
# c
import matplotlib.pyplot as plt
from glob import glob # to create a list of all the files present in a directory
import cv2
import random # to choose anything in random
import os
%matplotlib inline


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
# convo2d is used to read images and learn features from them.
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from IPython.display import SVG, Image
import tensorflow as tf 
print("tensorflow version: ",tf.__version__)

2024-03-07 15:53:31.473918: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.

To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tensorflow version:  2.12.0


## Preparing training and test data

In [3]:
img_size = 48
batch_size = 64
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_gen = datagen_train.flow_from_directory("/kaggle/input/fer2013/train/",target_size = (img_size,img_size),color_mode="grayscale",batch_size=batch_size,class_mode="categorical",shuffle=True)

datagen_validation = ImageDataGenerator(horizontal_flip=True)
train_validation = datagen_train.flow_from_directory("/kaggle/input/fer2013/test/",target_size = (img_size,img_size),color_mode="grayscale",batch_size=batch_size,class_mode="categorical",shuffle=True)


Found 28709 images belonging to 7 classes.

Found 7178 images belonging to 7 classes.


## Building Model layers

In [4]:
def Convolution(input_tensor, filters, kernel_size):
    x = Conv2D(filters=filters,kernel_size=kernel_size,padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.25)(x)

    return x

In [5]:
def Dense_f(input_tensor, nodes):
    x = Dense(nodes)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)

    return x

In [7]:
def model_fer(input_shape):
    inputs = Input(input_shape)
    conv_i = Convolution(inputs,64,(3,3))
    conv_ii = Convolution(conv_i,128,(5,5))
    conv_iii = Convolution(conv_ii,512,(3,3))
    conv_iv = Convolution(conv_iii,512,(3,3))

    flatten = Flatten()(conv_iv)

    dense_i = Dense_f(flatten,256)
    dense_ii = Dense_f(dense_i,512)
    output = Dense(7,activation="softmax")(dense_ii)
    model = Model(inputs=[inputs], outputs=[output])
    
    opt = Adam(lr=0.0005)

    model.compile(loss=['categorical_crossentropy'], optimizer=opt, metrics=['accuracy'])

    return model

In [8]:
model = model_fer((48,48,1))
model.summary()

Model: "model"

_________________________________________________________________

 Layer (type)                Output Shape              Param #   


 input_1 (InputLayer)        [(None, 48, 48, 1)]       0         

                                                                 

 conv2d (Conv2D)             (None, 48, 48, 64)        640       

                                                                 

 batch_normalization (Batch  (None, 48, 48, 64)        256       

 Normalization)                                                  

                                                                 

 activation (Activation)     (None, 48, 48, 64)        0         

                                                                 

 max_pooling2d (MaxPooling2  (None, 24, 24, 64)        0         

 D)                                                              

                                                                 

 dropout (Dropout)           (None, 24, 24, 6

## Building the model

In [10]:
epochs = 15
steps_per_epoch = train_gen.n//train_gen.batch_size
validation_steps = train_validation.n//train_validation.batch_size
checkpoint = ModelCheckpoint("model_weights.h5",monitor='val_accuracy', save_weights_only=True, mode='max', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=2, min_lr=0.00001)
callbacks = [checkpoint,reduce_lr]

## Removed the whole training part for avoiding unnecessary long epochs list

In [ ]:
history = model.fit(x= train_gen, steps_per_epoch= steps_per_epoch, epochs=epochs, validation_data=train_validation, validation_steps=validation_steps, callbacks=callbacks)


## Evaluating the model

In [12]:
# Model evaluation
model.evaluate(train_validation)

113/113 [==============================] - 6s 57ms/step - loss: 0.9863 - accuracy: 0.6325


[0.9863303303718567, 0.6324881315231323]

## saving the model architecture

In [15]:
model_json = model.to_json()
with open("model_a1.json","w") as json_file:
    json_file.write(model_json)